# Anomaly Detection - Azure Machine Learning service

Anomaly or outlier detection is essentially finding patterns that do not conform to expected behavior. There are several approaches to anomaly detection that are based on either statistical properties, clustering, classification, Principal Component Analysis (PCA), or subsampling. In this notebook we will look at an **autoencoder network** for anomaly or outlier detection. An autoencoder is a neural-net based, unsupervised learning model that is used to learn low-dimensional features that captures some structure underlying the high-dimensional input data.

## Lab Scenario

Groundwater level is an important metric, especially for agriculture states such as Iowa. One of the metrics [U.S. Geological Survey (USGS)](https://www.usgs.gov/) monitors is **depth to water level in feet below the land**. In this lab we will use a synthetic dataset that models certain scenarios for Polk County, Iowa. The three key weather-related metrics we will be using are:

- water-level (depth to water level in feet below the land)
- temperature
- humidity

The data is generated daily using realistic monthly averages for Polk County, Iowa, for the years 2017 – 2019. The data is generated daily for each of the 92 different sensors/locations within Polk County, Iowa – 3 years x 365 days x 92 sensors = 100,740 total sets of data.

We are going to be using 2 copies of the dataset for years 2017 -2019: 

1. Normal conditions for the county.
2. A gradual build up dry conditions in one of the regions in Polk County, Iowa over the months of June and July 2019.

The goal of this notebook is to develop an approach to monitor a group of sensors based on their proximity to each other to predict regional anomalies in real-time. We will be grouping the sensors in 6 different location-based clusters as identified by the previous notebook. Thus, in for model training, we will use cluster_id, along with month, temperature, humidity, and water level as our features.

To train an autoencoder model that learns the structures in the input data in this more complex scenario will need significant compute resources. Thus, in the notebook we will be leveraging the compute resources provided by **Azure Machine Learning service** for our model training.

After training the model, we will register the model with Azure Machine Learning Workspace, then package and deploy the model as a webservice to make predictions.

## Outline

1. **Setup**: Import required libraries, load the datasets, and create the Azure Machine Learning Workspace.

2. **Remotely Train the Autoencoder Network using the Azure ML Compute**: Create the Azure Machine Learning compute, and submit the training job.

3. **Establish criteria for anomalies**: Define approaches and thresholds for detecting anomalies based on the trained autoencoder model.

4. **Deploy Model to Azure Container Instance (ACI) as a Web Service**: Get the registered models from the Azure Machine Learning Workspace, create the scoring script, package and deploy the model for making predictions.

5. **Test Deployment**: Test the model deployment by making calls directly on the service object and also by calling the deployed webservice over HTTP.

## Setup

### Import required libraries 

In [ ]:
import azureml.core
from azureml.core import Experiment, Workspace, Run, Datastore
from azureml.core.compute import ComputeTarget
from azureml.core.model import Model
from azureml.train.dnn import TensorFlow
from azureml.train.estimator import Estimator
from azureml.widgets import RunDetails

print("Azure ML SDK version:", azureml.core.VERSION)

import pandas as pd
import numpy as np
import urllib.request
import os
import math
import timeit
from IPython.display import display, HTML, Image, SVG
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_colwidth', -1)
print("pandas version: {} numpy version: {}".format(pd.__version__, np.__version__))

import sklearn
from sklearn import preprocessing
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn_pandas import DataFrameMapper
from sklearn.cluster import KMeans
import pickle

import keras
import tensorflow
from keras.layers import Input, Dropout
from keras.layers.core import Dense
from keras.models import Sequential, load_model
from keras.models import Model as KModel # resolve name conflict with Azure Model
from keras import regularizers
from keras.models import model_from_json

from numpy.random import seed
from tensorflow import set_random_seed

print("keras version: {} tensorflow version: {} sklearn version: {}".format(keras.__version__, 
                                                                        tensorflow.__version__, sklearn.__version__))

%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns
from shapely.geometry import Point
import geopandas as gpd
from geopandas import GeoDataFrame

print('importing libraries done!')

**Helper method to display a pandas dataframe**

In [ ]:
def display_dataframe(df_in):
    s = df_in.style.set_properties(**{'text-align': 'left'})
    s.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    display(HTML(s.render()))

### Load the Datasets

In [ ]:
normal_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
              'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/anomaly_detection/normal_multi.xlsx')

gradual_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
               'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/anomaly_detection/gradual_multi.xlsx')

normal_df = pd.read_excel(normal_url)
gradual_df = pd.read_excel(gradual_url)

print('Size of dataset: {} rows'.format(len(normal_df)))
print('Done loading datasets!')

### Azure Machine Learning service setup
To begin, you will need to provide the following information about your Azure Subscription.

**If you are using your own Azure subscription, please provide names for subscription_id, resource_group, workspace_name and workspace_region to use.** Note that the workspace needs to be of type [Machine Learning Workspace](https://docs.microsoft.com/en-us/azure/machine-learning/service/setup-create-workspace).

**If an environment is provided to you be sure to replace XXXXX in the values below with your unique identifier.**

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*).

To get these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.

In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the Quick-Starts resource group

#Provide values for the existing Resource Group 
resource_group = "Quick-Starts-XXXXX" # <- replace XXXXX with your unique identifier

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "quick-starts-ws-XXXXX" # <- replace XXXXX with your unique identifier
workspace_region = "eastus" # <- region of your Quick-Starts resource group

### Create and connect to an Azure Machine Learning Workspace

Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [ ]:
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')

## Remotely Train the Autoencoder Network using the Azure ML Compute

In the following cells, you will *not* train the model against the data you just downloaded using the resources provided by Azure Notebooks. Instead, you will deploy an Azure ML Compute cluster that will download the data and use a trainings script to train the model. All of the training will be performed remotely with respect to this notebook. 

### Create AML Compute Cluster

Azure Machine Learning Compute is a service for provisioning and managing clusters of Azure virtual machines for running machine learning workloads. Let's create a new Aml Compute in the current workspace, if it doesn't already exist. We will run the model training jobs on this compute target. This will take couple of minutes to create.

In [ ]:
### Create AML CPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cluster_name = "amlcompute-ad"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC6',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### Create the training script

The training script downloads the data, defines and trains the autoencoder model. It saves the model files and also **registers** the trained models with Azure Machine Learning service.

In [ ]:
script_file_folder = './scripts'
script_file_name = 'train.py'
script_file_full_path = os.path.join(script_file_folder, script_file_name)
os.makedirs(script_file_folder, exist_ok=True)

In [ ]:
%%writefile $script_file_full_path
import os
import math
import timeit
import numpy as np
import pandas as pd
np.random.seed(437)

print("pandas version: {} numpy version: {}".format(pd.__version__, np.__version__))

import sklearn
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MinMaxScaler
from sklearn_pandas import DataFrameMapper
import pickle

import keras
import tensorflow
from keras.layers import Input, Dropout
from keras.layers.core import Dense 
from keras.models import Sequential, load_model
from keras.models import Model as KModel # resolve name conflict with Azure Model
from keras import regularizers
from keras.models import model_from_json
from numpy.random import seed
from tensorflow import set_random_seed

from azureml.core import Run
from azureml.core.model import Model

print("keras version: {} tensorflow version: {} sklearn version: {}".format(keras.__version__, 
                                                                            tensorflow.__version__, 
                                                                            sklearn.__version__))

print("Loading data file.")
normal_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
              'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/anomaly_detection/normal_multi.xlsx')

normal_df = pd.read_excel(normal_url)
print("Loading data file completed.")

feature_cols = ['cluster_id', 'month', 'temperature', 'humidity', 'water_level']
categorical = ['cluster_id', 'month']
numerical = ['temperature', 'humidity', 'water_level']

numeric_transformations = [([f], Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', MinMaxScaler())])) for f in numerical]
    
categorical_transformations = [([f], OneHotEncoder(handle_unknown='ignore', sparse=False)) for f in categorical]

transformations = categorical_transformations + numeric_transformations

clf = Pipeline(steps=[('preprocessor', DataFrameMapper(transformations))])

X = clf.fit_transform(normal_df[feature_cols])
X_train = X
np.random.shuffle(X_train)

seed(10)
set_random_seed(50)
act_func = 'elu'

input_ = Input(shape=(X_train.shape[1],))
x = Dense(100, activation=act_func)(input_)
x = Dense(50, activation=act_func)(x)
x = Dense(25, activation=act_func)(x)
encoder = Dense(12, activation=act_func, name='feature_vector')(x)
x = Dense(25, activation=act_func)(encoder)
x = Dense(50, activation=act_func)(x)
x = Dense(100, activation=act_func)(x)
output_ = Dense(X_train.shape[1], activation=act_func)(x)

model = KModel(input_, output_)
lr = 0.001
opt = keras.optimizers.Adam(lr=lr)
model.compile(loss='mse', optimizer=opt)

encoder_model = KModel(inputs=model.input, outputs=model.get_layer('feature_vector').output)
encoder_model.compile(loss='mse', optimizer='adam')

epochs = 200
batch_size = 16

print('')
print(model.summary())
print('')
print('lr: ', lr)
print('epochs: ', epochs)
print('batch_size: ', batch_size)
print('')

def schedule(epoch_number, current_lr):
    lr = current_lr
    if (epoch_number < 25):
        lr = 0.001
    if (epoch_number >= 25) & (epoch_number < 35):
        lr = 0.0005
    if (epoch_number >= 35) & (epoch_number < 50):
        lr = 0.0003
    if (epoch_number >= 50) & (epoch_number < 60):
        lr = 0.0001
    if (epoch_number >= 60) & (epoch_number < 70):
        lr = 0.00008
    if (epoch_number >= 70) & (epoch_number < 80):
        lr = 0.00006
    if (epoch_number >= 80) & (epoch_number < 90):
        lr = 0.00004
    if (epoch_number >= 90) & (epoch_number < 100):
        lr = 0.00002
    if (epoch_number >= 100) & (epoch_number < 125):
        lr = 0.000009
    if (epoch_number >= 125) & (epoch_number < 150):
        lr = 0.000007
    if (epoch_number >= 150) & (epoch_number < 175):
        lr = 0.000005
    if (epoch_number >= 175) & (epoch_number < 200):
        lr = 0.000001
    return lr

lr_sch = keras.callbacks.LearningRateScheduler(schedule, verbose=1)

print("Model training starting...")
start_time = timeit.default_timer()
history = model.fit(X_train, X_train, 
                    batch_size=batch_size, 
                    epochs=epochs, 
                    validation_split=0.2, 
                    callbacks=[lr_sch], 
                    verbose=2)
elapsed_time = timeit.default_timer() - start_time
print("Model training completed.")
print('Elapsed time (min): ', round(elapsed_time/60.0,0))

os.makedirs('./outputs', exist_ok=True)

# save the models
model.save(os.path.join('./outputs', 'anomaly_detection_multi_full_model.h5'))
encoder_model.save(os.path.join('./outputs', 'anomaly_detection_multi_encoder_model.h5'))

# save the feature preprocessor pipeline
pickle.dump(clf, open(os.path.join('./outputs', 'preprocessor.pkl'), 'wb'))

# save training history
with open(os.path.join('./outputs', 'history.txt'), 'w') as f:
    f.write(str(history.history))

print("Models saved in ./outputs folder")
print("Saving model files completed.")


# Register the Models
run = Run.get_context()

os.chdir("./outputs")

model_path = 'anomaly_detection_multi_full_model.h5'
model_name = 'anomaly_detector'
model_description = 'Autoencoder network for anomaly detection.'
model = Model.register(
    model_path=model_path,  # this points to a local file
    model_name=model_name,  # this is the name the model is registered as
    tags={"type": "autoencoder", "run_id": run.id},
    description=model_description,
    workspace=run.experiment.workspace
)

preprocessor_path = 'preprocessor.pkl'
preprocessor_name = 'preprocessor'
preprocessor_description = 'Feature preprocessor pipeline for anomaly detection.'
preprocessor = Model.register(
    model_path=preprocessor_path,  # this points to a local file
    model_name=preprocessor_name,  # this is the name the model is registered as
    tags={"type": "preprocessor", "run_id": run.id},
    description=preprocessor_description,
    workspace=run.experiment.workspace
)

os.chdir("..")

print("Model registered: {} \nModel Description: {} \nModel Version: {}".format(model.name, 
                                                                                model.description, 
                                                                                model.version))

print("Feature preprocessor pipeline registered: {} \nDescription: {} \nVersion: {}".format(
    preprocessor.name, 
    preprocessor.description, 
    preprocessor.version))

### Create the Keras estimator

In [ ]:
keras_est = TensorFlow(source_directory=script_file_folder,
                       compute_target=compute_target,
                       entry_script=script_file_name, 
                       conda_packages=['numpy==1.16.4', 'xlrd==1.2.0', 'pandas==0.25.1', 'scikit-learn==0.21.3'], 
                       pip_packages=['sklearn-pandas==1.8.0', 'keras==2.2.5'], 
                       framework_version='1.13')

### Submit the training run

The code pattern to submit a training run to Azure Machine Learning compute targets is always:

- Create an experiment to run.
- Submit the experiment.
- Wait for the run to complete.

#### Create the experiment

In [ ]:
experiment_name = 'anomaly-detection-lab3'
experiment = Experiment(ws, experiment_name)

#### Submit the experiment

Note that experiment run will perform the following:

- Build and deploy the container to Azure Machine Learning compute (~8 minutes)
- Execute the training script (~22 minutes)

If you change only the training script and re-submit, it will run faster the second time because the necessary container is already prepared so the time requried is just that for executing the training script.

In [ ]:
run = experiment.submit(keras_est, tags = {"type": "anomaly-detection"})

#### Monitor the Run

Using the azureml Jupyter widget, you can monitor the training run. This will approximately take around 30 minutes to complete. Once the training is completed you can then download the trained models locally by running the **Download the trained models** cell.

In [ ]:
RunDetails(run).show()

### Download the trained models

In [ ]:
# create an output folder in the current directory
os.makedirs('./outputs', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs'):
        output_file_path = os.path.join('./outputs', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

### Load the trained models

In [ ]:
# Load the trained models
model = load_model(os.path.join('./outputs', 'anomaly_detection_multi_full_model.h5'))
encoder_model = load_model(os.path.join('./outputs', 'anomaly_detection_multi_encoder_model.h5'))
clf = pickle.load(open(os.path.join('./outputs', 'preprocessor.pkl'), 'rb'))
print('Models loaded!')

## Establish Criteria for Anomalies

The autoencoder network is trained using normal data where it first compresses the input data and then reconstructs the input data. During training the network learns the interactions between various input variables under normal conditions and learns to reconstruct the input variables back to their original values. The reconstruction error is the error is reproducing back the original input values. We will be using `Mean Absolute Error` as our measure for the reconstruction error. The basic idea behind anomaly detection is that the reconstruction error using the trained network for anomalous inputs will be higher than what is typically observed with normal data. 

Thus, one of the parameters we need to understand is the **threshold for the reconstruction error** that identifies anomalous input data.

### Make predictions and compute reconstruction errors for the normal dataset

Next, we will make predictions on the normal dataset, compute the reconstruction error for individual set of inputs, and look that the upper and lower bounds for the reconstruction errors.

In [ ]:
feature_cols = ['cluster_id', 'month', 'temperature', 'humidity', 'water_level']
X_train = clf.transform(normal_df[feature_cols]) # Keep the order
X_pred = model.predict(X_train)
loss_mae = np.mean(np.abs(X_pred-X_train), axis = 1)
normal_df['loss_mae'] = loss_mae
stats = normal_df.loss_mae.describe()
print(('Max loss mae: {}').format(stats['max']))

### Visualize the reconstruction errors for the normal dataset

It appears that the threshold value of `0.001` is a reasonable cutoff to identify anomalous input data.

In [ ]:
# Setup upper_bound for anomalous reconstruction error
upper_bound = 0.001

In [ ]:
plt.figure(figsize=(7, 5))

upper_boundary = upper_bound * np.ones(len(normal_df.date.unique()))

plt.plot_date(normal_df.date, normal_df.loss_mae, markersize=0.5)
plt.plot(normal_df.date.unique(), upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

### Visualize the reconstruction errors for the gradual datasets

Make predictions on the gradual dataset, compute the reconstruction error for individual set of inputs. You will see a ramp up in the reconstruction error for some set of data points around June-August 2019.

In [ ]:
X_gradual = clf.transform(gradual_df[feature_cols])
X_gradual_pred = model.predict(X_gradual)
loss_mae_gradual = np.mean(np.abs(X_gradual-X_gradual_pred), axis = 1)

gradual_df['loss_mae'] = loss_mae_gradual

In [ ]:
plt.figure(figsize=(7, 5))

upper_boundary = upper_bound * np.ones(len(gradual_df.date.unique()))

plt.plot_date(gradual_df.date, gradual_df.loss_mae, markersize=0.5)
plt.plot(gradual_df.date.unique(), upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

### Visualize the reconstruction errors for the various region clusters in the gradual datasets

**Is there a lower error threshold we can monitor to detect the potential anomaly earlier in the time scale?**

In [ ]:
cluster_df = gradual_df.groupby(['date', 'cluster_id'])['water_level', 'loss_mae'].mean()
cluster_df.reset_index(drop=False, inplace=True)

In [ ]:
cluster_upper_bound = 0.0005

In [ ]:
plt.figure(figsize=(7, 5))

cluster_upper_boundary = cluster_upper_bound * np.ones(len(cluster_df.date.unique()))

plt.plot_date(cluster_df.date, cluster_df.loss_mae, markersize=0.5)
plt.plot(cluster_df.date.unique(), cluster_upper_boundary, color='r')

plt.xticks(fontsize=10, rotation=45);

plt.show()

## Deploy Anomaly Detector Model as Webservice

### Get Registered Models

In [ ]:
model_name = 'anomaly_detector'
preprocessor_name = 'preprocessor'

registered_model = Model(ws, name = model_name)
registered_preprocessor = Model(ws, name = preprocessor_name)

print(registered_model)
print()
print(registered_preprocessor)

### Create the Scoring Script

In [ ]:
%%writefile score.py
import json
import numpy as np
import pandas as pd
import os
import pickle
from keras.models import load_model
from azureml.core.model import Model

columns = ['cluster_id', 'month', 'temperature', 'humidity', 'water_level']

def init():
    global model
    global clf
    
    print('Get model paths...')
    model_path = Model.get_model_path('anomaly_detector')
    preprocessor_path = Model.get_model_path('preprocessor')
    print(model_path)
    print(preprocessor_path)

    print('Load models...')
    model = load_model(model_path)
    clf = pickle.load(open(preprocessor_path, 'rb'))
    print('Done loading models!')
    
def run(input_json):
    # Get predictions for each data point
    print('Create dataframe')
    inputs = json.loads(input_json)
    data_df = pd.DataFrame(np.array(inputs).reshape(-1, len(columns)), columns = columns)
    
    print('Make predictions')
    X = clf.transform(data_df)
    X_pred = model.predict(X)
    
    print('Calcuate loss_mae')
    loss_mae = np.mean(np.abs(X_pred-X), axis = 1)
    data_df['loss_mae'] = np.round(loss_mae, 5)
    
    print('Calcuate anomaly_std')
    upper_bound = 0.001    
    data_df['anomaly_std'] = data_df.loss_mae.apply(lambda x: True if x > upper_bound else False)
    
    # You can return any data type as long as it is JSON-serializable
    print('Return results')
    return {'loss_mae': data_df.loss_mae.values.tolist(), 'anomaly_std': data_df.anomaly_std.values.tolist()}

### Package Model

In [ ]:
# create a Conda dependencies environment file
print("Creating conda dependencies file locally...")
from azureml.core.conda_dependencies import CondaDependencies 
conda_packages = ['numpy==1.16.4', 'xlrd==1.2.0', 'pandas==0.25.1', 'scikit-learn==0.21.3']
pip_packages = ['azureml-sdk', 'sklearn-pandas==1.8.0', 'tensorflow==1.13.1', 'keras==2.2.5']
mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)

conda_file = 'dependencies.yml'
with open(conda_file, 'w') as f:
    f.write(mycondaenv.serialize_to_string())

runtime = 'python'

# create container image configuration
print("Creating container image configuration...")
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = 'score.py', 
                                                  runtime = runtime, 
                                                  conda_file = conda_file)

# create the image
image_name = 'anomaly-detector-image'

from azureml.core import Image
image = Image.create(name=image_name, 
                     models=[registered_model, registered_preprocessor], 
                     image_config=image_config, 
                     workspace=ws)

# wait for image creation to finish
image.wait_for_creation(show_output=True)

### Deploy Model to Azure Container Instance (ACI) as a Web Service

In [ ]:
from azureml.core.webservice import AciWebservice, Webservice

aci_name = 'anomaly-detector01'

aci_config = AciWebservice.deploy_configuration(
    cpu_cores = 1, 
    memory_gb = 1, 
    tags = {'name': aci_name}, 
    description = 'Autoencoder network for anomaly detection.')

service_name = 'anomaly-detector-service'

aci_service = Webservice.deploy_from_image(deployment_config=aci_config, 
                                           image=image, 
                                           name=service_name, 
                                           workspace=ws)

aci_service.wait_for_deployment(show_output=True)

## Test Deployment

### Make direct calls on the service object

In [ ]:
import json

# ['cluster_id', 'month', 'temperature', 'humidity', 'water_level']
data = [[1.0, 7.0, 79.6, 68.2, 15.7], [1.0, 7.0, 72.6, 68.2, 18.8]]

result = aci_service.run(json.dumps(data))
print('Predictions')
print(result)

### Make HTTP calls to test the deployed Web Service

In [ ]:
import requests

url = aci_service.scoring_uri
print('ACI Service: {} scoring URI is: {}'.format(aci_name, url))
headers = {'Content-Type':'application/json'}

response = requests.post(url, json.dumps(data), headers=headers)
print('Predictions')
print(response.text)